# Overview
- 数字特徴量を見てみる

# Const

In [69]:
NB = '003'
PATH_TRAIN = './../data/official/train.csv'
PATH_TEST = './../data/official/test.csv'
PATH_SAMPLE_SUBMITTION = './../data/official/atmaCup8_sample-submission.csv'
SAVE_DIR = f'../data/output_nb/nb{NB}/'

feat_train_only = ['JP_Sales', 'Global_Sales', 'NA_Sales', 'Other_Sales', 'EU_Sales']
feat_common = ['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher',
           'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Developer',
           'Rating']

# Import everything I need :)

In [17]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling as pdp
from lightgbm import LGBMRegressor 
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


# My function

In [6]:
def metric(y_true, y_pred):
    return mean_squared_log_error(y_true, y_pred) ** .5

# Preparation

set

In [7]:
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

<br>

load dataset

In [8]:
train = pd.read_csv(PATH_TRAIN)
test = pd.read_csv(PATH_TEST)
ss = pd.read_csv(PATH_SAMPLE_SUBMITTION)

# EDA

In [9]:
train.describe()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Count
count,8275.000000,8359.000000,8359.000000,8359.000000,8359.000000,8359.000000,3976.000000,3976.00000,3699.000000
mean,2006.393716,30.719943,16.067711,11.308889,5.241058,63.371815,69.187626,28.53999,180.262503
std,6.099621,104.349935,60.936947,41.215915,22.941531,199.394856,13.756481,20.42759,576.988465
min,1980.000000,0.000000,0.000000,0.000000,0.000000,1.000000,19.000000,4.00000,4.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,6.000000,61.000000,12.00000,11.000000
50%,2007.000000,8.000000,2.000000,0.000000,1.000000,18.000000,71.000000,24.00000,28.000000
75%,2010.000000,25.000000,12.000000,6.000000,4.000000,51.000000,79.000000,40.00000,100.000000
max,2017.000000,4136.000000,2896.000000,1022.000000,1057.000000,8253.000000,98.000000,113.00000,9851.000000


In [14]:
test.columns

Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher',
       'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Developer',
       'Rating'],
      dtype='object')

In [13]:
feat_train_only

['JP_Sales', 'Global_Sales', 'NA_Sales', 'Other_Sales', 'EU_Sales']

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8359 entries, 0 to 8358
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             8359 non-null   object 
 1   Platform         8359 non-null   object 
 2   Year_of_Release  8275 non-null   float64
 3   Genre            8359 non-null   object 
 4   Publisher        8359 non-null   object 
 5   NA_Sales         8359 non-null   int64  
 6   EU_Sales         8359 non-null   int64  
 7   JP_Sales         8359 non-null   int64  
 8   Other_Sales      8359 non-null   int64  
 9   Global_Sales     8359 non-null   int64  
 10  Critic_Score     3976 non-null   float64
 11  Critic_Count     3976 non-null   float64
 12  User_Score       4831 non-null   object 
 13  User_Count       3699 non-null   float64
 14  Developer        4870 non-null   object 
 15  Rating           4798 non-null   object 
dtypes: float64(4), int64(5), object(7)
memory usage: 1.0+ MB


In [16]:
train[feat_common].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8359 entries, 0 to 8358
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             8359 non-null   object 
 1   Platform         8359 non-null   object 
 2   Year_of_Release  8275 non-null   float64
 3   Genre            8359 non-null   object 
 4   Publisher        8359 non-null   object 
 5   Critic_Score     3976 non-null   float64
 6   Critic_Count     3976 non-null   float64
 7   User_Score       4831 non-null   object 
 8   User_Count       3699 non-null   float64
 9   Developer        4870 non-null   object 
 10  Rating           4798 non-null   object 
dtypes: float64(4), object(7)
memory usage: 718.5+ KB


In [19]:
train.head().T

,0,1,2,3,4
Name,LEGO Batman: The Videogame,LEGO Indiana Jones: The Original Adventures,LEGO Batman: The Videogame,Combat,LEGO Harry Potter: Years 5-7
Platform,Wii,Wii,PSP,2600,Wii
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Warner Bros. Interactive Entertainment,LucasArts,Warner Bros. Interactive Entertainment,Atari,Warner Bros. Interactive Entertainment
NA_Sales,180,151,56,117,69
EU_Sales,97,61,44,7,42
JP_Sales,0,0,0,0,0
Other_Sales,28,21,27,1,12
Global_Sales,306,234,128,125,124


In [27]:
train.User_Score.unique()

array(['7.9', '6.6', '7.4', nan, '7.8', '8', '8.3', '7', '6.8', 'tbd',
       '7.2', '6.5', '7.6', '4.1', '8.5', '6.1', '6', '6.3', '9.5', '6.4',
       '5.7', '7.3', '8.9', '2', '4.5', '8.7', '7.5', '6.7', '9.1', '2.2',
       '8.2', '9.2', '8.6', '8.4', '9', '9.4', '8.8', '8.1', '7.1', '7.7',
       '5.3', '4.6', '3.8', '2.5', '2.4', '6.9', '2.9', '5.4', '3.5',
       '4.9', '3.4', '9.3', '5.8', '6.2', '4', '5', '5.2', '4.8', '5.9',
       '5.1', '4.7', '5.6', '4.4', '3', '5.5', '3.6', '3.2', '3.1', '3.9',
       '2.8', '4.3', '4.2', '0.5', '3.7', '2.7', '9.7', '2.1', '1.2',
       '1.9', '1.7', '0.6', '3.3', '0', '1', '1.1', '1.8', '1.3', '0.9',
       '1.6'], dtype=object)

---> User_Scoreにtbdがある。なんだこれ。  

In [28]:
train.User_Score.value_counts()

tbd    1132
8       165
8.2     160
7.8     155
8.3     137
       ... 
1.6       1
0.6       1
0         1
9.7       1
1.9       1
Name: User_Score, Length: 88, dtype: int64

---> 調べると、tbd = To Be Determined らしい。nanに置き換えるか。

In [51]:
# tbdをnanに変えて、string-->floatにする
mask = train.User_Score.values == 'tbd'
train.User_Score[mask] = np.nan
train.User_Score = train.User_Score.values.astype(float)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
train.User_Score.unique()

array([7.9, 6.6, 7.4, nan, 7.8, 8. , 8.3, 7. , 6.8, 7.2, 6.5, 7.6, 4.1,
       8.5, 6.1, 6. , 6.3, 9.5, 6.4, 5.7, 7.3, 8.9, 2. , 4.5, 8.7, 7.5,
       6.7, 9.1, 2.2, 8.2, 9.2, 8.6, 8.4, 9. , 9.4, 8.8, 8.1, 7.1, 7.7,
       5.3, 4.6, 3.8, 2.5, 2.4, 6.9, 2.9, 5.4, 3.5, 4.9, 3.4, 9.3, 5.8,
       6.2, 4. , 5. , 5.2, 4.8, 5.9, 5.1, 4.7, 5.6, 4.4, 3. , 5.5, 3.6,
       3.2, 3.1, 3.9, 2.8, 4.3, 4.2, 0.5, 3.7, 2.7, 9.7, 2.1, 1.2, 1.9,
       1.7, 0.6, 3.3, 0. , 1. , 1.1, 1.8, 1.3, 0.9, 1.6])